In [1]:
import pandas as pd
import numpy as np
import os
from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR 
#pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

Generates association rules for the results found in the projects history fifth slice.

In [2]:
#df = pd.read_excel("historical_join.xlsx", keep_default_na=False)
df = pd.read_excel(HISTORICAL_FILE_JOIN, keep_default_na=False)

len(df)
df.head()

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df2 = df.copy()
df2

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31111,liferay,liferay-portal,Enterprise Resource Planning,fcd4cd6526c80613eca69927effe931bb6a91444,8209,2024-10-07 12:05:23+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31112,liferay,liferay-portal,Enterprise Resource Planning,cc0ab8103195e543056639c8c2883fcc187c33a2,8210,2024-10-07 17:58:58+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31113,liferay,liferay-portal,Enterprise Resource Planning,f75082029372ab28694650bd3fe9fae4d548351e,8211,2024-10-11 04:36:52+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31114,liferay,liferay-portal,Enterprise Resource Planning,a80a679a86d2867f985bafa510d9d12f1bf051a7,8212,2024-10-11 16:53:22+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0


Grouping the lines to identify which projects that don't have DB clues

In [4]:
df2 = df2.groupby(["owner", "name"]).sum()
df2

,,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,HyperSQL,CouchDB,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
owner,name,,,,,,,,,,,,,,,,,,,,,
Activiti,Activiti,1540,0,0,0,0,0,0,0,13,0,...,0,0,0,0,0,0,0,0,0,0
Alluxio,alluxio,5886,0,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
AlmasB,FXGL,105,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AntennaPod,AntennaPod,703,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AnySoftKeyboard,AnySoftKeyboard,703,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xuxueli,xxl-job,120,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
yacy,yacy_search_server,5151,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zaproxy,zaproxy,666,0,0,0,0,0,0,0,36,0,...,0,0,0,0,0,0,0,0,0,0


Reseting the index column

In [5]:
df2 = df2.reset_index()
df2

,owner,name,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,1540,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alluxio,alluxio,5886,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
2,AlmasB,FXGL,105,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AntennaPod,AntennaPod,703,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AnySoftKeyboard,AnySoftKeyboard,703,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,xuxueli,xxl-job,120,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
358,yacy,yacy_search_server,5151,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359,zaproxy,zaproxy,666,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
360,zendesk,maxwell,105,0,0,0,0,0,0,0,...,0,0,0,0,0,7,0,0,0,0


New dataframe filtering only lines that fifth version projects

In [6]:
# Função para selecionar o commit da metade
def select_half_commit(group):
    middle_index = len(group) // 2  # Calcula o índice do meio
    return group.iloc[middle_index-1]  # Seleciona o commit do meio

# Ordena os commits dentro de cada projeto (caso não estejam ordenados)
df_sorted = df.sort_values(by=['name', 'part_commit'])

# Aplica a função para pegar o commit da metade em cada grupo de projeto
df10 = df_sorted.groupby('name').apply(select_half_commit).reset_index(drop=True)

df10


,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,d2c8230955ee79c53450f5d1b58682a9e85bbcc9,27,2013-01-29 16:56:43+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CellularPrivacy,Android-IMSI-Catcher-Detector,Security,814033630a829e1a258598982b481f85bd6f883f,6,2015-01-01 15:16:52+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Blankj,AndroidUtilCode,Software Development,c584fb12027505d55ecc27f962c720edc72e8fc1,5,2017-08-28 09:38:47+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AntennaPod,AntennaPod,Media,d5db62c70b867f21a74e158e5f7e860c46952062,18,2017-05-31 15:09:20+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AnySoftKeyboard,AnySoftKeyboard,Other,15bb43e4c828d9c990478ebd3f7b1d6d2950acd0,18,2015-05-07 21:05:07+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,apache,zeppelin,Data Management,9d40013a994ca65f99831a416fb235d15ee17fdb,21,2017-03-20 08:11:38+00:00,False,0,0,0,...,0,1,0,0,0,1,0,1,0,0
357,openzipkin,zipkin,Monitoring,5880cd979f12ad0ab5bf1948a3c3f473e2b75978,13,2018-06-11 06:49:23+00:00,False,0,0,0,...,0,1,0,0,0,0,0,0,0,0
358,apache,zookeeper,Infrastructure Management,74d8363cd562d4c082ae4bc9120d693def2ddb6b,12,2014-05-15 01:35:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359,zxing,zxing,Software Development,8198d32f530d4c23db3859553f67ecf77aec7201,18,2011-11-16 23:51:06+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Removing projects that don't have DB clues the principal dataframe 

In [7]:
#df2 = df2.iloc[ : , 1:]
#df3[(df3 == 0).all(axis=1)].index
df2 = df2.drop(df2[(df2.iloc[ : , 8:] == 0).all(axis=1)].index)
df2

,owner,name,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,1540,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alluxio,alluxio,5886,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
4,AnySoftKeyboard,AnySoftKeyboard,703,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Athou,commafeed,465,0,0,0,0,0,0,0,...,0,0,0,0,0,18,0,0,0,0
7,Atmosphere,atmosphere,1326,0,0,0,0,0,0,0,...,0,0,0,0,0,26,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,wildfly,wildfly,27495,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
357,xuxueli,xxl-job,120,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
358,yacy,yacy_search_server,5151,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359,zaproxy,zaproxy,666,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Removing projects that don't have DB clues in the fifth version 

In [8]:
#df3 = df[(df['COMMITS'] == 5)] #mudar aqui tmb

def select_half_commit(group):
    middle_index = len(group) // 2  # Calcula o índice do meio
    return group.iloc[middle_index-1]  # Seleciona o commit do meio

# Ordena os commits dentro de cada projeto (caso não estejam ordenados)
df_sorted = df.sort_values(by=['name', 'part_commit'])

# Aplica a função para pegar o commit da metade em cada grupo de projeto
df3 = df_sorted.groupby('name').apply(select_half_commit).reset_index(drop=True)

#df3.drop(["PROJECTS", "COMMITS"], axis=1, inplace=True)
df3 = df3.drop(df3[(df3.iloc[:,8:] == 0).all(axis=1)].index) # deletando as linhas que possuem valor 0 em todas as colunas
df3

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,d2c8230955ee79c53450f5d1b58682a9e85bbcc9,27,2013-01-29 16:56:43+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,AxonFramework,AxonFramework,Infrastructure Management,75d533daf261ae88370bb3cc39840b9332a30627,17,2016-09-07 20:27:30+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,opensourceBIM,BIMserver,Data Management,4d5e50a0ef8f47d996f75dc3e2cc5e21a4ae87bd,21,2014-02-11 20:15:48+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,BroadleafCommerce,BroadleafCommerce,Enterprise Resource Planning,4400a9d5603be89f2e60718e45c563f08581964b,33,2013-04-11 21:46:56+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,SpigotMC,BungeeCord,Game,5a638f22902e920722283db73e1296e28121972e,9,2014-04-16 01:01:41+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,xuxueli,xxl-job,Automation,d025f7152749f3a54663206aa188561204de45c3,7,2017-12-22 12:54:54+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
354,yacy,yacy_search_server,Infrastructure Management,3f746be5d4dc8ffbd65e589c4d0dbed8c1bdf9b4,50,2008-11-06 10:07:53+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
355,zaproxy,zaproxy,Security,d6bf06efe1a09d0ac091457eee014e71fad786e4,18,2014-03-07 22:07:57+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
356,apache,zeppelin,Data Management,9d40013a994ca65f99831a416fb235d15ee17fdb,21,2017-03-20 08:11:38+00:00,False,0,0,0,...,0,1,0,0,0,1,0,1,0,0


Merging dataframe grouping and dataframe only fifth version with DB clues

In [9]:
#df3.merge(df4, on=['PROJECTS'], how='outer', suffixes=['', '_'], indicator=True)
df_mer= pd.merge(df2, df3, on= 'name', how='outer', suffixes=['', '_'], indicator=True)
df_mer

,owner,name,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,...,Cassandra_,Couchbase_,Etcd_,PostGIS_,Redis_,Join_Ignite-Sql_,Join_Ignite-NoSql_,Join_Virtuoso-Sql_,Join_Virtuoso-NoSql_,_merge
0,Activiti,Activiti,1540.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
1,Alluxio,alluxio,5886.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,AnySoftKeyboard,AnySoftKeyboard,703.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,Athou,commafeed,465.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,both
4,Atmosphere,atmosphere,1326.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,xuxueli,xxl-job,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
227,yacy,yacy_search_server,5151.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
228,zaproxy,zaproxy,666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
229,zendesk,maxwell,105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both


Filtering diference in the fifth version

In [10]:
df5 = df_mer[(df_mer['_merge'].str.startswith('left'))]
df5

,owner,name,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,...,Cassandra_,Couchbase_,Etcd_,PostGIS_,Redis_,Join_Ignite-Sql_,Join_Ignite-NoSql_,Join_Virtuoso-Sql_,Join_Virtuoso-NoSql_,_merge
1,Alluxio,alluxio,5886.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,AnySoftKeyboard,AnySoftKeyboard,703.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Atmosphere,atmosphere,1326.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7,FasterXML,jackson-databind,1596.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
11,JSQLParser,JSqlParser,153.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
16,M66B,NetGuard,630.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
18,NationalSecurityAgency,ghidra,1326.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
24,SeleniumHQ,selenium,36315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
38,apache,avro,861.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
40,apache,bookkeeper,528.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


Merging dataframe only lines that fifth version projects and dataframe only lines in the fifth version and not DB clues

In [11]:
df_mer2= pd.merge(df10, df5.iloc[:,:2], on= 'name', how='outer', suffixes=['', '_'], indicator=True)
df_mer2

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,owner_,_merge
0,Activiti,Activiti,Enterprise Resource Planning,d2c8230955ee79c53450f5d1b58682a9e85bbcc9,27,2013-01-29 16:56:43+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
1,CellularPrivacy,Android-IMSI-Catcher-Detector,Security,814033630a829e1a258598982b481f85bd6f883f,6,2015-01-01 15:16:52+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
2,Blankj,AndroidUtilCode,Software Development,c584fb12027505d55ecc27f962c720edc72e8fc1,5,2017-08-28 09:38:47+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
3,AntennaPod,AntennaPod,Media,d5db62c70b867f21a74e158e5f7e860c46952062,18,2017-05-31 15:09:20+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
4,AnySoftKeyboard,AnySoftKeyboard,Other,15bb43e4c828d9c990478ebd3f7b1d6d2950acd0,18,2015-05-07 21:05:07+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,AnySoftKeyboard,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,apache,zeppelin,Data Management,9d40013a994ca65f99831a416fb235d15ee17fdb,21,2017-03-20 08:11:38+00:00,False,0,0,0,...,0,0,0,1,0,1,0,0,NaN,left_only
357,openzipkin,zipkin,Monitoring,5880cd979f12ad0ab5bf1948a3c3f473e2b75978,13,2018-06-11 06:49:23+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
358,apache,zookeeper,Infrastructure Management,74d8363cd562d4c082ae4bc9120d693def2ddb6b,12,2014-05-15 01:35:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,apache,both
359,zxing,zxing,Software Development,8198d32f530d4c23db3859553f67ecf77aec7201,18,2011-11-16 23:51:06+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only


Filtering only lines in the fifth version and not DB clues

In [12]:
df6 = df_mer2[(df_mer2['_merge'].str.startswith('bo'))]
df6

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,owner_,_merge
4,AnySoftKeyboard,AnySoftKeyboard,Other,15bb43e4c828d9c990478ebd3f7b1d6d2950acd0,18,2015-05-07 21:05:07+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,AnySoftKeyboard,both
6,asciidocfx,AsciidocFX,Media,d9cf49761881a0f91149cfe333c87f9958ddbbda,6,2015-07-14 17:14:50+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,asciidocfx,both
14,stanfordnlp,CoreNLP,Artificial Intelligence,463913e3c287b14c1231bae531ec023eb9b4f7ec,76,2014-07-02 17:57:45+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,stanfordnlp,both
22,JSQLParser,JSqlParser,Software Development,1786e215474e8da7d03399483a962dc627d9d342,8,2018-09-08 23:44:37+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,JSQLParser,both
27,M66B,NetGuard,Security,144b1e3a6728005218b5c6498be7fd277b9059f2,17,2016-02-21 11:39:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,M66B,both
33,opentripplanner,OpenTripPlanner,Data Management,f18804858258a880fc6b322afcf644b4448750de,35,2014-08-28 16:50:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,opentripplanner,both
58,Alluxio,alluxio,Data Management,8d93e3684c6374f80e9645bcf04806056e0497b5,54,2018-04-30 16:39:17+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,Alluxio,both
66,Atmosphere,atmosphere,Network,848429f4d1fcdedfc476dcc0be38c10ebeac3fc1,25,2013-03-14 18:47:54+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,Atmosphere,both
69,apache,avro,Data Management,d6cf7c46c20973f4e2e4ef41c064a40fb616994d,20,2020-01-02 20:37:41+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,apache,both
77,bisq-network,bisq,Cryptocurrency,557a09e37ed29c515623806ac76b4039f392197b,26,2018-04-04 19:41:54+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,bisq-network,both


Concatening dataframes

In [13]:
df_concat2 =  pd.concat([df3, df6]).drop_duplicates()
df_concat2

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,owner_,_merge
0,Activiti,Activiti,Enterprise Resource Planning,d2c8230955ee79c53450f5d1b58682a9e85bbcc9,27,2013-01-29 16:56:43+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
7,AxonFramework,AxonFramework,Infrastructure Management,75d533daf261ae88370bb3cc39840b9332a30627,17,2016-09-07 20:27:30+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
8,opensourceBIM,BIMserver,Data Management,4d5e50a0ef8f47d996f75dc3e2cc5e21a4ae87bd,21,2014-02-11 20:15:48+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
9,BroadleafCommerce,BroadleafCommerce,Enterprise Resource Planning,4400a9d5603be89f2e60718e45c563f08581964b,33,2013-04-11 21:46:56+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
11,SpigotMC,BungeeCord,Game,5a638f22902e920722283db73e1296e28121972e,9,2014-04-16 01:01:41+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,apache,struts,Software Development,d89c7186a12e9ec2d93b566fe83e92d5254e6eea,24,2009-08-07 18:24:45+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,apache,both
329,swagger-api,swagger-core,Software Development,739b6ba199ed7529231f09f4ec27de00a18b492e,7,2015-10-12 05:00:36+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,swagger-api,both
348,eclipse-vertx,vert.x,Infrastructure Management,45faf61b3b3839cd59956dc7164125f873d22043,18,2018-06-13 20:07:46+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,eclipse-vertx,both
350,oracle,visualvm,Monitoring,e9a6d0605382852f7e29bb3a6c1911a53d7dcd25,22,2010-03-23 10:38:04+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,oracle,both


In [14]:
#df2 = df2.astype(np.uint8)

In [15]:
df_concat2.drop(["owner","name", "domain", "sha1", "part_commit", "date_commit", "isLast", "owner_", "_merge"], axis=1, inplace=True)
df_concat2

,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,HyperSQL,CouchDB,Riak KV,Neo4j,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
348,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
350,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ASSOCIATION RULES

In [16]:
frq_items = apriori(df_concat2, min_support = 0.021, use_colnames = True)

In [17]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)

In [18]:
# Sorting the rules by the lift measure 
rules = rules.sort_values('lift', ascending =False)
print(rules.head())

                                              antecedents  \
166234              (HyperSQL, Firebird, SQLite, IBM DB2)   
168091                    (PostgreSQL, SapHana, Informix)   
168179  (SQLite, IBM DB2, Oracle, PostgreSQL, HyperSQL...   
168132                                (Firebird, SapHana)   
168130                                (SapHana, Informix)   

                                              consequents  antecedent support  \
166234                     (MySQL, H2, SapHana, Informix)            0.030435   
168091        (SQLite, Oracle, HyperSQL, MySQL, Firebird)            0.021739   
168179                                (Firebird, SapHana)            0.030435   
168132  (SQLite, Oracle, PostgreSQL, HyperSQL, Informi...            0.021739   
168130  (SQLite, Oracle, PostgreSQL, HyperSQL, MySQL, ...            0.021739   

        consequent support   support  confidence       lift  leverage  \
166234            0.021739  0.021739    0.714286  32.857143  0.021078   


In [19]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
166234,"(HyperSQL, Firebird, SQLite, IBM DB2)","(MySQL, H2, SapHana, Informix)",0.030435,0.021739,0.021739,0.714286,32.857143,0.021078,3.423913
168091,"(PostgreSQL, SapHana, Informix)","(SQLite, Oracle, HyperSQL, MySQL, Firebird)",0.021739,0.030435,0.021739,1.000000,32.857143,0.021078,inf
168179,"(SQLite, IBM DB2, Oracle, PostgreSQL, HyperSQL...","(Firebird, SapHana)",0.030435,0.021739,0.021739,0.714286,32.857143,0.021078,3.423913
168132,"(Firebird, SapHana)","(SQLite, Oracle, PostgreSQL, HyperSQL, Informi...",0.021739,0.030435,0.021739,1.000000,32.857143,0.021078,inf
168130,"(SapHana, Informix)","(SQLite, Oracle, PostgreSQL, HyperSQL, MySQL, ...",0.021739,0.030435,0.021739,1.000000,32.857143,0.021078,inf
...,...,...,...,...,...,...,...,...,...
68,(HyperSQL),(HBase),0.273913,0.069565,0.021739,0.079365,1.140873,0.002684,1.010645
183,(Google Cloud Datastore),(MySQL),0.104348,0.456522,0.052174,0.500000,1.095238,0.004537,1.086957
182,(MySQL),(Google Cloud Datastore),0.456522,0.104348,0.052174,0.114286,1.095238,0.004537,1.011220
1144,"(HyperSQL, MySQL)",(Google Cloud Datastore),0.191304,0.104348,0.021739,0.113636,1.089015,0.001777,1.010479


In [20]:
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequente_len'] = rules['consequents'].apply(lambda x: len(x))

Filtering only the rules of size 2, that is, 1 antecedent and 1 consequent. 

In [21]:
rules_filter = rules[(rules['antecedent_len'] == 1) & (rules['consequente_len'] == 1)]

Showing all filtered rules.

In [22]:
rules_filter

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequente_len
20,(Firebird),(SapHana),0.034783,0.030435,0.021739,0.625000,20.535714,0.020681,2.585507,1,1
21,(SapHana),(Firebird),0.030435,0.034783,0.021739,0.714286,20.535714,0.020681,3.378261,1,1
43,(Informix),(Firebird),0.043478,0.034783,0.030435,0.700000,20.125000,0.028922,3.217391,1,1
42,(Firebird),(Informix),0.034783,0.043478,0.030435,0.875000,20.125000,0.028922,7.652174,1,1
92,(Couchbase),(Neo4j),0.034783,0.034783,0.021739,0.625000,17.968750,0.020529,2.573913,1,1
...,...,...,...,...,...,...,...,...,...,...,...
153,(DynamoDB),(Oracle),0.091304,0.256522,0.030435,0.333333,1.299435,0.007013,1.115217,1,1
69,(HBase),(HyperSQL),0.069565,0.273913,0.021739,0.312500,1.140873,0.002684,1.056126,1,1
68,(HyperSQL),(HBase),0.273913,0.069565,0.021739,0.079365,1.140873,0.002684,1.010645,1,1
183,(Google Cloud Datastore),(MySQL),0.104348,0.456522,0.052174,0.500000,1.095238,0.004537,1.086957,1,1


In [23]:
HISTORICAL_RULES_FILE = RESOURCE_DIR + os.sep + 'historical_rulesv5.xlsx' #'seq_patterns/testes/historical_rules_commit5_197reg_021.xlsx'
rules_filter = rules_filter.to_excel(HISTORICAL_RULES_FILE, index=False)